# Azure AI Agents with LangChain Integration

This notebook demonstrates how to use Azure AI Agents with LangChain through the `langchain-azure-ai` integration, powered by the Azure AI Projects SDK.

## Prerequisites

1. An Azure AI Foundry project
2. A deployed model (e.g., gpt-4o-mini)
3. Appropriate authentication credentials
4. Environment variables set:
   - `PROJECT_CONNECTION_STRING`

## New Implementation

This updated implementation uses the **Azure AI Projects SDK** (`azure.ai.projects.AIProjectClient`) instead of the lower-level Azure AI Agents SDK. This provides:

- Better integration with Azure AI Foundry projects
- Simplified authentication using connection strings
- More robust tool handling and serialization
- Better support for file uploads and management

## Install Required Packages

For local development, you have a few options:

### Option 1: Install in editable mode (Recommended for development)
Run this in your terminal from the `libs/azure-ai` directory:
```bash
pip install -e .
```

### Option 2: Install individual dependencies
If you want to install just the dependencies without the full package:
```bash
pip install azure-ai-agents azure-identity langchain-core
```

### Option 3: Use Python path (for this notebook only)
The next cell will set up the Python path to use the local code directly.

## Local Development Setup

If you're running this notebook from the langchain-azure repository, you need to set up the environment to use the local build that includes the new Azure AI Agents integration.

## Install Required Packages

For local development, you have a few options:

### Install in editable mode (Recommended for development)
Run this in your terminal from the `libs/azure-ai` directory:
```bash
pip install .
```

## Import Required Modules

## Set Up Environment Variables

To use Azure AI Agents, you need to get your project connection string from Azure AI Foundry:

1. Go to [Azure AI Foundry](https://ai.azure.com)
2. Open your project
3. Go to **Project settings** → **General**
4. Copy the **Connection string**

The connection string should look like:
```
https://your-project.region.ai.azure.com/api/projects/your-project-name
```

Make sure these are set in your environment:

In [ ]:
import sys
import os
from pathlib import Path

# Get the path to the langchain_azure_ai package in the current repo
current_dir = Path.cwd()
# Navigate to the libs/azure-ai directory regardless of where we are
azure_ai_path = None

# Try multiple possible locations
possible_paths = [
    current_dir / "langchain_azure_ai",  # If we're in the azure-ai directory
    current_dir.parent / "langchain_azure_ai",  # If we're in docs subdirectory
    current_dir / "libs" / "azure-ai" / "langchain_azure_ai",  # If we're in repo root
    current_dir.parent.parent / "langchain_azure_ai",  # If we're nested deeper
]

for path in possible_paths:
    if path.exists() and (path / "__init__.py").exists():
        azure_ai_path = path.parent
        break

if azure_ai_path:
    # Add to Python path if not already there
    azure_ai_str = str(azure_ai_path)
    if azure_ai_str not in sys.path:
        sys.path.insert(0, azure_ai_str)
    print(f"✓ Added local package path: {azure_ai_path}")
    
    # Verify the package is accessible
    try:
        import langchain_azure_ai
        print(f"✓ Successfully imported langchain_azure_ai from: {langchain_azure_ai.__file__}")
        
        # Check if Azure AI Agents is available
        try:
            from langchain_azure_ai.azure_ai_agents import AzureAIAgentsService
            print("✓ Azure AI Agents integration is available in the local build!")
        except ImportError as e:
            print(f"✗ Azure AI Agents not found: {e}")
            
    except ImportError as e:
        print(f"✗ Could not import langchain_azure_ai: {e}")
else:
    print("✗ Could not find langchain_azure_ai package in expected locations")
    print("   Make sure you're running this notebook from within the langchain-azure repository")
    print("   Current directory:", current_dir)


In [15]:
# Try to import Azure and LangChain modules
from azure.identity import DefaultAzureCredential
from langchain_azure_ai.azure_ai_agents import AzureAIAgentsService
import langchain_azure_ai

# Import other required modules
from langchain_core.messages import HumanMessage
from langchain_core.callbacks import CallbackManager
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
import os

# Set up environment variables for Azure AI Projects
# Replace with your actual connection string
os.environ["PROJECT_CONNECTION_STRING"] = ""
os.environ["PROJECT_ENDPOINT"] = ""
os.environ["MODEL_DEPLOYMENT_NAME"] = ""


## Basic Usage

Let's start with a basic example of creating an Azure AI Agent and using it with LangChain:

**Note**: The Azure AI Projects SDK uses `project_connection_string` for authentication instead of separate endpoint and model parameters.

In [17]:
# Create an Azure AI Agents service using Azure AI Projects SDK
agent_service = AzureAIAgentsService(
    credential=DefaultAzureCredential(),
    endpoint=os.environ["PROJECT_ENDPOINT"],  # Use the project endpoint
    model_name="gpt-4.1",  # Use a model that's available in your project
    agent_name="langchain-demo-agent",
    instructions="You are a helpful AI assistant that provides clear and concise answers.",
)

print("Agent service created successfully!")

Agent service created successfully!


In [19]:
# Test basic generation
response = agent_service.invoke("What is the capital of France?")
print(f"Response: {response}")

Response: The capital of France is Paris.


## Multiple Prompts

You can process multiple prompts at once:

In [20]:
# Generate responses for multiple prompts
prompts = [
    "What is 2 + 2?",
    "Name one programming language.",
    "What color is the sky?"
]

results = agent_service.generate(prompts)

for i, generation_list in enumerate(results.generations):
    print(f"Prompt {i+1}: {prompts[i]}")
    print(f"Response: {generation_list[0].text}\n")

Prompt 1: What is 2 + 2?
Response: 2 + 2 = 4

Prompt 2: Name one programming language.
Response: Python is a programming language.

Prompt 3: What color is the sky?
Response: The sky appears blue during the day because of the way Earth's atmosphere scatters sunlight. At sunrise and sunset, it often looks red, orange, or pink due to the angle of the sun and longer passage through the atmosphere. At night, the sky is usually dark or black, except for the stars and moon.



## Integration with LangChain Chains

The Azure AI Agents service works seamlessly with LangChain chains:

In [10]:
# Create a prompt template
prompt = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} in exactly 3 bullet points."
)

# Create a chain
chain = prompt | agent_service | StrOutputParser()

# Use the chain
result = chain.invoke({"topic": "machine learning"})
print(f"Machine Learning Explanation:\n{result}")

Machine Learning Explanation:
- Machine learning is a field of artificial intelligence where computers learn from data to make predictions or decisions without explicit programming.  
- It involves feeding large amounts of data into algorithms, which identify patterns and improve their performance over time.  
- Common applications include image recognition, language translation, and recommendation systems.


In [11]:
# Try with different topics
topics = ["artificial intelligence", "cloud computing", "data science"]

for topic in topics:
    result = chain.invoke({"topic": topic})
    print(f"\n{topic.title()}:")
    print(result)
    print("-" * 50)


Artificial Intelligence:
- Artificial intelligence (AI) is the development of computer systems capable of performing tasks that typically require human intelligence, such as learning, reasoning, and problem-solving.
- AI uses algorithms and large datasets to recognize patterns, make decisions, and improve its performance over time without explicit human programming for each task.
- Common AI applications include virtual assistants, image and speech recognition, autonomous vehicles, and recommendation systems.
--------------------------------------------------

Cloud Computing:
- Cloud computing delivers computing services (such as storage, servers, databases, and software) over the internet, allowing on-demand access to resources.
- It eliminates the need for physical hardware and enables scalability, flexibility, and cost-effectiveness by charging only for what you use.
- Services are typically categorized as Infrastructure as a Service (IaaS), Platform as a Service (PaaS), and Softw

## Advanced Configuration

You can customize the agent with various parameters. The Azure AI Agents service supports the following configuration options:

**Supported Parameters:**
- `temperature`: Controls randomness in the output (0.0 to 1.0)
- `top_p`: Controls diversity via nucleus sampling (0.0 to 1.0)  
- `response_format`: Specifies the format that the model must output
- `tools`: List of tools to provide to the agent
- `tool_resources`: Tool resources for the agent
- `metadata`: Custom metadata for the agent

**Note:** Some parameters like `max_completion_tokens`, `max_prompt_tokens`, etc. are not supported by the current Azure AI Agents SDK.

In [21]:
# Create an agent with custom configuration
custom_agent = AzureAIAgentsService(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
    model_name=os.environ["MODEL_DEPLOYMENT_NAME"],  # Use model_name instead of model
    agent_name="custom-configured-agent",
    instructions="You are a creative writing assistant. Write in a poetic style.",
    temperature=0.8,  # Higher temperature for more creativity
    agent_description="A creative writing assistant with poetic flair"
)

# Test the custom agent
creative_response = custom_agent.invoke("Describe a sunset over the ocean.")
print(f"Creative Response:\n{creative_response}")

Creative Response:
The horizon drinks the sun with slow abandon,  
Brushstrokes of gold unfurl across the waves,  
A trembling path of fire, softly laden  
With whispers of the secrets daylight gave.  

Clouds blush in apricot and rose-tint splendor,  
Their fringes melting into violet sighs;  
Salt air grows cool, the world grows tender,  
As sapphire shadows gather in the skies.  

Seagulls wheel through the final honeyed glimmer,  
Their silhouettes like brushwork on the air—  
The distant sun a fading, molten shimmer  
That slips beneath the water’s gentle stare.  

Night hushes in, the day sings its surrender,  
Leaving the ocean gleaming with remembered amber.


## Direct Client Access

For advanced use cases, you can access the underlying Azure AI Agents client:

## Tool Usage with Azure AI Agents

Azure AI Agents support various tools including code interpreter, file search, and function calling. Here are examples of how to use these tools with the LangChain integration using the Azure AI Projects SDK.

### Code Interpreter Tool

The code interpreter tool allows agents to write and execute Python code, analyze data, create visualizations, and work with files.

In [ ]:
!pip install pandas 

In [12]:
# First, let's create some sample data for the code interpreter to work with
import pandas as pd
from pathlib import Path 
import tempfile
import os

# Create sample sales data
data = {
    'month': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun'],
    'sales': [12000, 15000, 18000, 14000, 22000, 25000],
    'region': ['North', 'South', 'East', 'West', 'North', 'South']
}

df = pd.DataFrame(data)

# Create a CSV file in the current working directory
csv_path = Path.cwd() / "sample_sales_data.csv"
df.to_csv(csv_path, index=False, encoding='utf-8-sig')  # The code interpreter requires utf-8-sig 

print(f"Created sample data file → {csv_path}")
print("\nSample data:")
print(df)

Created sample data file → /workspaces/langchain-azure/libs/azure-ai/docs/sample_sales_data.csv

Sample data:
  month  sales region
0   Jan  12000  North
1   Feb  15000  South
2   Mar  18000   East
3   Apr  14000   West
4   May  22000  North
5   Jun  25000  South


In [ ]:
# Import required modules for tools using Azure AI Projects SDK
from azure.ai.agents.models import CodeInterpreterTool, FilePurpose, FileSearchTool, MessageRole
# from azure.ai.projects import AIProjectClient

try:
    agents_client = AzureAIAgentsService(
    credential=DefaultAzureCredential(),
    endpoint=os.environ["PROJECT_ENDPOINT"],  # Use the project endpoint
    # model_name="gpt-4.1",  # Use a model that's available in your project
    )._create_client()

    # # Upload the file for the code interpreter
    print("Uploading file for code interpreter...")
    uploaded_file = agents_client.files.upload_and_poll(
        file_path=str(csv_path),
        purpose=FilePurpose.AGENTS  # Specify the purpose for the file
    )

    print(f"✓ Uploaded file, file ID: {uploaded_file.id}")

    # Create code interpreter tool with the uploaded file
    code_interpreter = CodeInterpreterTool(file_ids=[uploaded_file.id])

    print(code_interpreter.definitions)
    
    print(code_interpreter.resources)

    agent = agents_client.create_agent(
                model="gpt-4.1",
                name="code-interpreter-agent",
                instructions="""You are a data analyst agent. 
                Analyze the provided data and create visualizations 
                when helpful. Use Python code to explore and understand the data.""",
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources
            )
    print(f"Created agent, agent ID: {agent.id}")

    thread = agents_client.threads.create()
    print(f"Created thread, thread ID: {thread.id}")

    print(f"✓ Created AzureAIAgentsService with code interpreter tool")

    # Create a message
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="create a pie chart with the data showing sales by region and show it to me as a png image.",
    )
    print(f"Created message, message ID: {message.id}")

    run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # [START get_messages_and_save_files]
    messages = agents_client.messages.list(thread_id=thread.id)
    print(f"Messages: {messages}")

    for msg in messages:
        breakpoint()
        # Save every image file in the message
        for img in msg.image_contents:
            file_id = img.image_file.file_id
            file_name = f"{file_id}_image_file.png"
            agents_client.files.save(file_id=file_id, file_name=file_name, target_dir="/workspaces/langchain-azure/libs/azure-ai/docs/")
            print(f"Saved image file to: {Path.cwd() / file_name}")

        # Print details of every file-path annotation
        for ann in msg.file_path_annotations:
            print("File Paths:")
            print(f"  Type: {ann.type}")
            print(f"  Text: {ann.text}")
            print(f"  File ID: {ann.file_path.file_id}")
            print(f"  Start Index: {ann.start_index}")
            print(f"  End Index: {ann.end_index}")
    # [END get_messages_and_save_files]

    last_msg = agents_client.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Please check your Azure AI Projects configuration and try again.")
    import traceback
    traceback.print_exc()


In [ ]:
# Create a persistent thread for multi-turn conversation
thread = agents_client.threads.create()
print(f"Created thread: {thread.id}")

# Have a multi-turn conversation
conversation = [
    "Hello! I'm learning about Python.",
    "What are the key features of Python?",
    "Can you give me an example of a Python function?"
]

for user_message in conversation:
    # Create message
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_message
    )
    
    # Run the agent
    run = agents_client.runs.create_and_process(
        thread_id=thread.id,
        agent_id=agent.id
    )
    
    # Get the latest response
    messages = agents_client.messages.list(thread_id=thread.id)
    
    # Find the latest assistant message
    # ItemPaged is iterable directly, no need for .data
    for msg in messages:
        if msg.role == "assistant" and msg.text_messages:
            assistant_response = msg.text_messages[0].text.value
            break
    
    print(f"\nUser: {user_message}")
    print(f"Assistant: {assistant_response}")
    print("-" * 80)

# Clean up the thread
agents_client.threads.delete(thread.id)
print("\nThread deleted.")

## Async Operations

The service also supports async operations:

In [14]:
import asyncio
from azure.identity import DefaultAzureCredential

async def async_example():
    # Create async agent
    async_agent = AzureAIAgentsService(
        endpoint=os.environ["PROJECT_ENDPOINT"],
        credential=DefaultAzureCredential(),
        model_name=os.environ["MODEL_DEPLOYMENT_NAME"],  # Use model_name instead of model
        agent_name="async-demo-agent",
        instructions="You are an async AI assistant."
    )
    
    try:
        # Async generation
        response = await async_agent.ainvoke("What are the benefits of asynchronous programming?")
        print(f"Async Response: {response}")
        
        # Async multiple prompts
        prompts = ["Hello", "How are you?", "Goodbye"]
        results = await async_agent.agenerate(prompts)
        
        print("\nAsync Multiple Responses:")
        for i, generation_list in enumerate(results.generations):
            print(f"{prompts[i]}: {generation_list[0].text}")
    
    finally:
        # Clean up
        await async_agent.adelete_agent()
        await async_agent.aclose()          # closes aiohttp session
        
# Run the async example
await async_example()

Async Response: Asynchronous programming offers several benefits, especially in applications that require handling multiple tasks or I/O operations efficiently. Here are the main advantages:

1. **Improved Responsiveness**  
   Asynchronous code allows a program to perform other tasks while waiting for operations (like file I/O or network requests) to complete. This keeps user interfaces (UIs) and services responsive and prevents blocking.

2. **Efficient Resource Utilization**  
   Asynchronous operations free up system resources by not blocking threads during wait times. This allows a single thread to manage multiple I/O-bound operations, reducing the need for many threads and saving memory and CPU.

3. **Scalability**  
   Asynchronous programming enables applications (especially servers) to handle many concurrent connections or operations efficiently. For example, asynchronous web servers can serve thousands of clients without requiring a separate thread per client.

4. **Better Th

## Cleanup

Always remember to clean up your agents when you're done:

In [35]:
# Clean up all the agents we created
try:
    agent_service.delete_agent()
    print("Deleted main agent")
except:
    print("Main agent already deleted or not found")

try:
    custom_agent.delete_agent()
    print("Deleted custom agent")
except:
    print("Custom agent already deleted or not found")

print("\nCleanup completed!")

Deleted main agent
Deleted custom agent

Cleanup completed!
Deleted custom agent

Cleanup completed!


## Summary

This notebook demonstrated the updated Azure AI Agents integration with LangChain using the **Azure AI Projects SDK**:

### Key Improvements

1. **Azure AI Projects SDK**: Now uses `azure.ai.projects.AIProjectClient` instead of the lower-level `azure-ai-agents` SDK
2. **Simplified Authentication**: Uses project connection strings for easier setup
3. **Better Tool Support**: Improved handling of tools like code interpreter with proper serialization
4. **Robust File Management**: Better file upload and management capabilities
5. **LangChain Integration**: Seamless integration with LangChain chains and workflows

### What was demonstrated:

1. **Basic Usage**: Creating and using Azure AI Agents with LangChain
2. **Multiple Prompts**: Processing multiple prompts efficiently  
3. **Chain Integration**: Using agents in LangChain chains and workflows
4. **Advanced Configuration**: Customizing agent behavior with parameters
5. **Tool Usage**: Working with code interpreter for data analysis
6. **Async Operations**: Working with async/await patterns
7. **Cleanup**: Properly managing agent resources

### Next Steps

- Explore file search capabilities
- Implement custom function calling
- Build multi-tool agents
- Integrate with more complex LangChain workflows

The Azure AI Agents integration provides a powerful way to combine the capabilities of Azure AI Projects with the flexibility of LangChain, enabling you to build sophisticated AI applications with ease.